In [2]:
import json
import requests
import numpy as np
import pandas as pd 
from tqdm import tqdm
from time import sleep
from dotenv import dotenv_values

from geopy.geocoders import Nominatim

In [3]:
def get_vacancies_id(header:dict, param:dict=None, URL:str='https://api.hh.ru/vacancies', per_page:int=100, time_delay:float=0)->json:
    sleep(time_delay)
    return requests.get(url=URL, headers=header, params=param).json()

In [4]:
JOB_TITLE = 'Аналитик данных'

USER_AGENT = dotenv_values('.env')['USER_AGENT']
BASE_URL='https://api.hh.ru/vacancies'

PER_PAGE = 100

header = {'User-Agent':USER_AGENT}
param = {'text': JOB_TITLE,
          'search_field': 'name',
          'page': 0,
          'per_page': PER_PAGE,
          'only_with_salary': True,
          'locale': 'RU'}

In [5]:
resp = get_vacancies_id(header, param, per_page=PER_PAGE)

VAC_CNT = resp['found']
PAGES_CNT = resp['pages']
current_page = resp['page']
VAC_CNT, PAGES_CNT, current_page

(195, 2, 0)

In [6]:
vac_ids = []
current_page = 0
PAGES_CNT = 999

while current_page <= PAGES_CNT-1:
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']
    PAGES_CNT = resp['pages']

    for item in resp['items']:
        vac_ids.append(item['id'])

    param['page'] = current_page + 1
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']


print("It's all OK" if len(set(vac_ids)) == VAC_CNT else f"Smt went WRONG\n{len(set(vac_ids))}___{VAC_CNT}")

Smt went WRONG
196___195


In [7]:
def get_skills_str(key_skills:list)->str:
    '''Функция преобразования списка словарей требуемых скилов в строку'''
    result = []
    for skill in key_skills:
        result.append(skill.get('name'))
    return '; '.join(result)

In [18]:
data_list = []

for vac_id in tqdm(vac_ids, desc='Получение описания вакансий...'):
    try:
        vac_annote = get_vacancies_id(header=header, URL=BASE_URL+f'/{vac_id}', time_delay=0.4)


        address = vac_annote.get('address')
        employer = vac_annote.get('employer')
        salary = vac_annote.get('salary')
        salary_range = vac_annote.get('salary_range', {})
        mode = salary_range.get('mode', {}) if salary_range else {}
        prof_roles = vac_annote.get('professional_roles', [])
        
        name = vac_annote.get('name', '').lower()
        if any(word in name for word in ['стажер', 'стажёр', 'интерн', 'помощник', 'intern', 'trainee']):
            grade = 'Intern'
        elif any(word in name for word in ['младший', 'junior', 'джуниор', 'начинающий']):
            grade = 'Junior'
        elif any(word in name for word in ['мидл', 'middle', 'средний', 'mid-level']):
            grade = 'Middle'
        elif any(word in name for word in ['сеньор', 'senior', 'старший', 'ведущий', 'опытный']):
            grade = 'Senior'
        elif any(word in name for word in ['тимлид', 'teamlead', 'руководитель', 'lead', 'главный']):
            grade = 'Team Lead'
        else:
            grade = 'Middle'


        row = {
            'vac_id': vac_annote.get('id'),
            'vac_name': vac_annote.get('name'),
            'grade':grade,
            'city': vac_annote.get('area', {}).get('name'),
            'geo': f"[{address.get('lat')}, {address.get('lng')}]" if address else None,
            'published_at': vac_annote.get('published_at'),
            'archived': vac_annote.get('archived'),
            'employer_id': employer.get('id') if employer else None,
            'emp_name': vac_annote.get('employment', {}).get('name'),
            'addres': address.get('raw') if address else None,
            'is_accredited': employer.get('accredited_it_employer') if employer else None,
            'is_trusted': employer.get('trusted') if employer else None,
            'salary_from': salary.get('from') if salary else None,
            'salary_to': salary.get('to') if salary else None,
            'currency': salary.get('currency') if salary else None,
            'gross': salary.get('gross') if salary else None,
            'mode_name': mode.get('name'),
            'frequency': mode.get('frequency'),
            'prof_role': prof_roles[0].get('name') if prof_roles else None,
            'schedule_name': vac_annote.get('schedule', {}).get('name'),
            'insider_interview': vac_annote.get('insider_interview'),
            'response_letter_required': vac_annote.get('response_letter_required'),
            'experience': vac_annote.get('experience', {}).get('name'),
            'key_skills': get_skills_str(vac_annote.get('key_skills')) if vac_annote.get('key_skills') else None,
            'has_test': vac_annote.get('has_test'),
            'description': vac_annote.get('description'),
            'url':vac_annote.get('alternate_url')
        }
        data_list.append(row)
        
    except Exception as e:
        print(f'Error processing vac_id {vac_id}: {e}')

# Создаем DataFrame одним вызовом
full_df = pd.DataFrame(data_list)
print('Success: Parsing done')

Получение описания вакансий...:   0%|          | 0/196 [00:00<?, ?it/s]

Получение описания вакансий...: 100%|██████████| 196/196 [01:53<00:00,  1.72it/s]

Success: Parsing done


In [20]:
def get_geopoints(city:str):
    geolocator = Nominatim(user_agent="geoapi")
    try:
        location = geolocator.geocode(f"{city}, Россия")
    except Exception as e:
        return None
    finally:
        return (location.latitude, location.longitude)


In [21]:
mask = full_df['geo'].isna()
full_df.loc[mask, 'city'].apply(
    lambda x: get_geopoints(x) if pd.notnull(x) else (None, None)
)

1       (59.9606739, 30.1586551)
6       (56.5254053, 66.4787932)
7       (59.9606739, 30.1586551)
11       (55.625578, 37.6063916)
14       (55.625578, 37.6063916)
22       (55.625578, 37.6063916)
38      (56.3264816, 44.0051395)
43       (55.625578, 37.6063916)
44        (56.205997, 95.706787)
47       (55.625578, 37.6063916)
56      (52.5934637, 62.6172418)
64      (42.8769526, 74.5969359)
69      (55.0505685, 60.1087125)
72       (55.625578, 37.6063916)
74       (57.099061, 93.3343983)
80       (55.625578, 37.6063916)
81        (54.991375, 73.371529)
84      (55.7997662, 37.9373707)
85      (45.0351532, 38.9772396)
93      (47.2216548, 39.7096061)
103     (55.1674213, 59.6792625)
114      (55.625578, 37.6063916)
119       (56.313618, 55.406574)
120     (51.6605982, 39.2005858)
121     (59.9606739, 30.1586551)
127    (50.2600417, 127.5337378)
128      (55.625578, 37.6063916)
136      (54.328047, 48.3960145)
144     (42.8769526, 74.5969359)
146      (55.625578, 37.6063916)
150      (

In [11]:
full_df.loc[full_df.geo.isna()]

,vac_id,vac_name,grade,city,geo,published_at,archived,employer_id,emp_name,addres,...,frequency,prof_role,schedule_name,insider_interview,response_letter_required,experience,key_skills,has_test,description,url
1,122540411,Аналитик данных,Middle,Санкт-Петербург,None,2025-07-08T16:42:47+0300,False,4334427,Полная занятость,None,...,None,"BI-аналитик, аналитик данных",Полный день,None,False,От 3 до 6 лет,None,False,<p><strong>О проекте</strong></p> <p>Крупный к...,https://hh.ru/vacancy/122540411
6,122588313,Аналитик данных (Middle),Middle,Астана,None,2025-07-09T16:52:49+0300,False,11234839,Полная занятость,None,...,None,"BI-аналитик, аналитик данных",Полный день,None,False,От 1 года до 3 лет,None,False,<p><strong>Требования:</strong></p> <ul> <li> ...,https://hh.ru/vacancy/122588313
7,122638366,Аналитик данных,Middle,Санкт-Петербург,None,2025-07-10T16:37:17+0300,False,41989,Полная занятость,None,...,None,"BI-аналитик, аналитик данных",Удаленная работа,None,False,От 1 года до 3 лет,None,False,<p><strong>Задачи и обязанности:</strong></p> ...,https://hh.ru/vacancy/122638366
11,121516507,Аналитик данных,Middle,Москва,None,2025-07-10T12:49:18+0300,False,11587720,Полная занятость,None,...,None,"Оператор ПК, оператор базы данных",Удаленная работа,None,False,От 1 года до 3 лет,None,False,<p><strong>О компании</strong></p> <p>Мы — Bam...,https://hh.ru/vacancy/121516507
14,122551459,Аналитик данных/Junior аналитик e-com,Junior,Москва,None,2025-07-09T07:57:42+0300,False,10970452,Полная занятость,None,...,None,Другое,Удаленная работа,None,True,От 1 года до 3 лет,None,False,"<p><strong>Hello, world! </strong></p> <p>Мы -...",https://hh.ru/vacancy/122551459
22,122637990,Аналитик данных,Middle,Москва,None,2025-07-10T16:31:07+0300,False,11169602,Полная занятость,None,...,None,"BI-аналитик, аналитик данных",Удаленная работа,None,False,От 3 до 6 лет,Аналитическое мышление; A/B тесты; REST API; S...,False,"<p>Мы - продуктовая fintech компания, разработ...",https://hh.ru/vacancy/122637990
38,121084720,Аналитик данных,Middle,Нижний Новгород,None,2025-07-09T10:54:16+0300,False,562595,Полная занятость,None,...,None,Другое,Полный день,None,False,От 1 года до 3 лет,None,False,<strong>Обязанности:</strong> <ul> <li>Проводи...,https://hh.ru/vacancy/121084720
43,122546286,Разработчик / Аналитик базы данных,Middle,Москва,None,2025-07-08T20:31:11+0300,False,11491099,Полная занятость,None,...,None,Системный аналитик,Полный день,None,False,От 1 года до 3 лет,Исследовательский анализ данных; Базы данных; ...,False,"<p>Привет, тебе выпала уникальная возможность ...",https://hh.ru/vacancy/122546286
44,122514028,Аналитик данных,Middle,Канск,None,2025-07-08T10:22:19+0300,False,136475,Полная занятость,None,...,None,Аналитик,Полный день,None,False,Нет опыта,None,False,<p><strong>ОБЯЗАННОСТИ:</strong></p> <ul> <li>...,https://hh.ru/vacancy/122514028
47,120719059,Аналитик данных,Middle,Москва,None,2025-07-08T10:17:26+0300,False,654795,Полная занятость,None,...,None,"BI-аналитик, аналитик данных",Полный день,None,False,От 1 года до 3 лет,None,False,<p>Мы – одно из крупнейших в РФ агентств в сфе...,https://hh.ru/vacancy/120719059


--- 

In [ ]:
full_df.to_csv('resources/full_df.csv')
print('Success: DataFrame saved')

Success: DataFrame saved


In [16]:
%pip install gsheets

   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.2 kB ? eta -:--:--
   ------------------------------------- -- 92.2/98.2 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/13.7 MB 1.6 MB/s eta 0:00:09
   ---------------------------------------- 0.1/13.7 MB 1.2 MB/s eta 0:00:12
    --------------------------------------- 0.2/13.7 MB 1.6 MB/s eta 0:00:09
   - -------------------------------------- 0.4/13.7 MB 2.1 MB/s eta 0:00:07
   - -------------------------------------- 0.6/13.7 MB 2.2 MB/s eta 0:00:06
   - -------------------------------------- 0.7/13.7 MB 2.2 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/13.7 MB 2.2 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/13.7 MB 2.2 MB/s eta 0:00:06
   -- -----------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
path_to_gdisk = r'E:\Obsidian\YandexDisk'

full_df.to_excel(path_to_gdisk+'\\my.xlsx')

